In [1]:
import BubbleCount.image_preprocess as image_preprocess
import BubbleCount.csv_helpers as csv_helpers
from tqdm import tqdm
import csv

# Helper function
def simplify_target_name(target):
    temp = target.split('_')[1:-1]
    temp = '_'.join(temp)
    serial_number = temp.split('.')[-1]
    batch_name = '.'.join(temp.split('.')[0:-2])

    return f"{batch_name}_{serial_number}"

In [2]:
from BubbleCount.counting_model import CountingPipe
model = CountingPipe()  # Only really need the count_hybrid() function in this notebook

===> Using CPU mode.



### Arguments:
`raw_img_dir` - Folder containing uncropped images to be cropped to AOI <br>
`target_path` - Folder for images cropped to AOI <br>
`result_path` - .csv file containing count predictions <br>
`output_dir` - Folder for output images (Density plot, Graph, Overlaid)


In [3]:
args = {
    "raw_img_dir": "./2025/Images/SEN10_1.8_6_300-400", # images to be cropped
    "target_path": "./2025/Targets", # cropped images
    "result_path": "./2025/Outputs/out.csv", # csv containing counts
    "output_dir": "./2025/Outputs/", # output images
    


    # Do not modify
    "sample_path": "./Exemplars/",
    "model_path": "./data/pretrainedModels/FamNet_Save1.pth",
}

### Image cropping
`region` - y1, x1, y2, x2 coordiantes for a bounding box to crop raw images <br><br>

Takes images from `raw_img_dir` and outputs cropped iamges in `target_path`.

In [ ]:
# Crop images to AOI
region=[65, 770, 1090, 970]
image_preprocess.crop_to_interest(image_dir=args["raw_img_dir"],region=region,output_dir=args["target_path"])

### Predict and output

Takes all images from `target_path`, overlays exemplar bubbles from `sample_path`, then predicts and generates density plots, graphs, and the overlaid image to `output_dir`.

In [4]:
result_to_csv = []

# Load exemplars and targets
Exemplars = image_preprocess.load_exemplars_from_directory(args["sample_path"], reverse_bbox=False)
Targets = image_preprocess.load_target_images_from_directory(args["target_path"])

sample_image = Exemplars[1]

csv_helpers.backup_and_clear_csv(args["result_path"])

with open(args["result_path"], 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Exemplar", "num_exemps", "rev_bbox", "Target", "Count"])

    total = len(Targets)
    count = 0
    
    for target_image in tqdm(Targets, desc="Processing images"):
        num_exemps = 4
        # Creating the hybrid
        hybrid, hybrid_boxes = image_preprocess.insert_cropped(sample_image['image'], target_image['image'], sample_image['box'], num_exemps)
        target_name = f"{simplify_target_name(target_image['file_name'])}_{num_exemps}exemps"

        # Count and output into separate files
        hybrid_count = model.count_hybrid_and_visualize_separate(hybrid, hybrid_boxes, sample_image["file_name"], target_name,output_directory=args["output_dir"])

        result_to_csv.append([sample_image['file_name'], num_exemps, False, target_name, hybrid_count])

        # append to csv
        writer.writerows(result_to_csv)

        count = count + 1

print(f"The counts are saved to {args['result_path']}.")

Processing images:   0%|          | 0/600 [00:00<?, ?it/s]c:\Users\Brandon\Desktop\fellowship2025\BubbleCount-with-FamNet\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Brandon\Desktop\fellowship2025\BubbleCount-with-FamNet\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Processing images:   1%|          | 6/600 [00:07<13:04,  1.32s/it]c:\Users\Brandon\Desktop\fellowship2025\BubbleCount-with-FamNet\utils.py:398: RuntimeWarning: More than 20 figures have been opened. Figures created through 

The counts are saved to ./2025/Outputs/out.csv.


### Video creation tool
`dir` - Category of images to create video from. Options: `Density`, `Graphs`, `Overlaid`. <br>
`fps` - Frames per second of final video. Default: 10<br>

Takes images from `output_dir` and the specified `dir` and creates a video with the specified `fps`.

In [ ]:
import cv2
import os

# Generate videos from images
dir = "Overlaid" # "Density" or "Graphs" or "Overlaid"
fps = 10

input_folder = f'{args["output_dir"]}{dir}/'

print(input_folder)

# Get the list of image files in the input folder
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')])

# Read the first image to get its dimensions
first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
height, width, _ = first_image.shape

# Create a VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec for the output video file
video = cv2.VideoWriter(f"{input_folder}video.mp4", fourcc, fps, (width, height))

# Iterate over each image and write it to the video
for image_file in image_files:
    print(image_file)
    image_path = os.path.join(input_folder, image_file)
    frame = cv2.imread(image_path)
    video.write(frame)

# Release the video writer and close the video file
video.release()
cv2.destroyAllWindows()
